# **Ex1 - Unsupervised learning**

## Names and IDs

1.   oren fix 315666479
2.   nitzan bakalchuk 318595386


In this assignemnt you will practice unsupervised methods we saw in class, and specifically running K-means and visualizing the data using PCA.

In this assignment you will learn a few more things:

1.   Load local files
2.   Load data from Kaggle
3. Use Scikit-learn K-means
4. Use Scikit-learn PCA
5. Some visulaization
6. Evaluate the performance of the clustering using Elbow methods, Siouhette analysis and accuracy (as we have true labels)

Note:
* Read the complete task before implementing.
* Reuse code, write functions.

## Import All Packages
Add all imports needed for this notebook to run

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
#from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Text Clustering And Dimension Reduction

In this exercise you will cluster fake news using `k-means` and visualize the clustering using PCA.

**Import the Fake News Dataset from Kaggle**

Navigate to https://www.kaggle.com. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account) and select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below and click the upload button to upload kaggle.json to your Colab runtime.

After uploading the kaggle.json the fake news dataset will be copy to the enviroment in the '/content' directory. You will see the two files 'Fake.csv' and 'True.csv'.

For more about the dataset you can read [here](https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset).

In [2]:
from google.colab import files

# upload kaggle.json file using user prompt
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# download the dataset
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

# extract the files
!unzip '/content/fake-and-real-news-dataset.zip'

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 63 bytes
fake-and-real-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/fake-and-real-news-dataset.zip
replace Fake.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace True.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


## 1. Read the csv files and create one single dataframe (5 points)

*   Create a dataframe which is the join of the two files 'Fake.csv' and 'True.csv'.
*   Extract the 'text' column from each dataframe (droping title, subject and date columns).
*   Create a single dataframe containing a text column and a label column (fake=0, real=1).

In [17]:
# Implement here
# Example for reading one file. It will work if the previous step was successful.
face_df = pd.read_csv('/content/Fake.csv')
df_fake = pd.read_csv("/content/Fake.csv")
df_true = pd.read_csv("/content/True.csv")
df_fake['class']=0
df_true['class']=1
frames = [df_fake,df_true]
df_join = pd.concat(frames)
df = df_join[['text', 'class']]
df

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
21412,BRUSSELS (Reuters) - NATO allies on Tuesday we...,1
21413,"LONDON (Reuters) - LexisNexis, a provider of l...",1
21414,MINSK (Reuters) - In the shadow of disused Sov...,1
21415,MOSCOW (Reuters) - Vatican Secretary of State ...,1


## 2. Cluster the samples into 2 clusters (real and fake) (15 points)

*   Generate TF-IDF features by applying the TfidfVectorizer preprocessor using 1000 features (`max_features=1000`) and remove English stop words.
* Scale the data.
*    Apply k-Means algorithm on the TF-IDF features using n_clusters=2.

In [19]:
english_stop_words = set(stopwords.words('english'))
tfidf = TfidfVectorizer(stop_words=english_stop_words,max_features=1000)
fiting = tfidf.fit_transform(df['text'])
Scaler = preprocessing.StandardScaler()
x_scaled = Scaler.fit_transform(fiting.toarray())

df_tfidf = pd.DataFrame(x_scaled, columns=tfidf.get_feature_names())



df_tfidf

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,000,10,100,11,12,13,14,15,16,17,...,would,wrong,wrote,year,years,yes,yet,york,young,youtube
0,-0.312974,-0.271,-0.210303,1.506896,-0.201248,-0.164631,-0.158476,-0.215370,-0.160032,-0.16126,...,0.197545,2.103623,-0.242712,10.121715,2.461539,-0.165118,-0.296579,-0.292620,-0.193335,-0.143699
1,-0.312974,-0.271,-0.210303,-0.196805,-0.201248,-0.164631,-0.158476,-0.215370,-0.160032,-0.16126,...,-0.717719,-0.195666,-0.242712,-0.514505,-0.452914,-0.165118,-0.296579,4.380094,-0.193335,-0.143699
2,-0.312974,-0.271,-0.210303,-0.196805,-0.201248,-0.164631,-0.158476,-0.215370,-0.160032,-0.16126,...,-0.717719,-0.195666,-0.242712,0.061199,-0.452914,-0.165118,-0.296579,-0.292620,-0.193335,-0.143699
3,-0.312974,-0.271,-0.210303,-0.196805,-0.201248,-0.164631,-0.158476,-0.215370,-0.160032,-0.16126,...,0.974256,-0.195666,-0.242712,0.359377,-0.452914,-0.165118,-0.296579,-0.292620,-0.193335,-0.143699
4,-0.312974,-0.271,-0.210303,-0.196805,-0.201248,-0.164631,-0.158476,-0.215370,-0.160032,-0.16126,...,-0.717719,-0.195666,-0.242712,-0.514505,-0.452914,-0.165118,2.897630,-0.292620,2.947139,-0.143699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,2.088203,-0.271,-0.210303,1.894094,2.195845,-0.164631,-0.158476,2.004468,2.753297,-0.16126,...,0.967194,-0.195666,-0.242712,0.355729,0.739358,-0.165118,-0.296579,-0.292620,-0.193335,-0.143699
44894,-0.312974,-0.271,-0.210303,-0.196805,-0.201248,-0.164631,-0.158476,-0.215370,-0.160032,-0.16126,...,-0.717719,-0.195666,-0.242712,3.505648,-0.452914,-0.165118,-0.296579,-0.292620,-0.193335,-0.143699
44895,-0.312974,-0.271,-0.210303,-0.196805,-0.201248,-0.164631,-0.158476,-0.215370,-0.160032,-0.16126,...,-0.717719,-0.195666,-0.242712,1.228803,4.323960,-0.165118,-0.296579,-0.292620,-0.193335,-0.143699
44896,-0.312974,-0.271,-0.210303,-0.196805,-0.201248,-0.164631,-0.158476,-0.215370,-0.160032,-0.16126,...,-0.717719,-0.195666,-0.242712,1.235417,-0.452914,-0.165118,-0.296579,-0.292620,-0.193335,-0.143699


In [3]:
|english_stop_words = set(stopwords.words('english'))

NameError: ignored

In [34]:
df_text = str(df['text'])

In [16]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-e3e4e05234c9>", line 6, in <module>
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/series.py", line 4357, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/apply.py", line 1043, in apply
    return self.apply_standard()
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/apply.py", line 1101, in apply_standard
    convert=self.convert_dtype,
  File "pandas/_libs/lib.pyx", line 2859, in pandas._libs.lib.map_infer
  File "<ipython-input-16-e3e4e05234c9>", line 6, in <lambda>
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() 

KeyboardInterrupt: ignored

In [1]:
stop = stopwords.words("english")

# def remove_english_stopwords_func(text):
#   t = [token for token in text if token.lower() not in stop ]
#   return t  
# textt = []  
# for ind in df_fake.index:
#   for i in df['text'][ind]:
#     splitt = i.split(" ")
#   s = remove_english_stopwords_func(splitt)  
#   textt = textt + s
# print(textt)



NameError: ignored

In [ ]:
uniqueWords = set(bagOfWordsA).union(set(bagOfWordsB))

In [30]:
x = df["text"]
y = df["class"]
max_features=1000
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## 3. Visualize using scatter plot (15 points)
The data is high dimensional, so for visualization purpose, we will perform a dimensionality reduction using PCA.

* Apply PCA
*   Visualize the clustering in 2d using first two PCs.
*   Visualize the clustering in 3d using first three PCs.

Notes:

*   In clustering visualization it's important to also visualize the centroids.
* Visualize using matplotlib scatter function. It can be used to plot 2D or 3D scatter plots.
* Color the points according to the true labels.

In [ ]:
# Implement here

## 4. Performance evaluation (15 points)
### How did our clustering perform against the real labels?

We do have the true lables (fake/real), but we don't know which cluster correspond to which label. Therefore, we check the two options:

*   Define cluster 1 as fake and cluster 2 as real. What is the accuracy?
*   Define cluster 1 as true and cluster 2 as fake. What is the accuracy?

In [ ]:
# Implement here

## 5. Find optimal `k` (10 points)
What is the best k for clustering?

* Cluster using a range of `k` (up to 20) and compare the SSD and the Silhouette values for every k.
* Plot SSD vs. k and Silhouette score vs. k.

Notes:
* You can get the SSD of a clustering using the `inertia_` attribute of the model.
* Silhouette score using `silhouette_score` function from `sklearn.metrics`. This function accept the model and the data.
* Computing Silhouette may takes long time. Estimate the Silhouette using a sample of 300 samples uisng the argument `sample_size=300`.

In [ ]:
# Implement here

### 6. What is the optimal k for clustering? (10 points)

* Explain.
* If optimal k!=2 what can be a good explanation for this?

Your answer here

# 7. PCA then k-means (10 points)

1.   List item
2.   List item


This time, lets try to change the order.
1. Generate TF-IDF 1000 features
2. Run PCA (using all dimenssions)
3. Run k-means
4. Plot 2D and 3D scatter plots
5. Estimate the accuracy according to true labels.
6. Explain how is it compared to the previous approach, where clustering is performed w/o PCA.

Note:
* When you compute the PCA, don't limit to first PCs, but use all of them.

In [ ]:
""" Answer """

# MNIST clustering (10 points)
Redo the same analysis, but this time using the MNIST data set. Notice that this time there are actually 10 'true' clusters.

1. What is the accuracy of the clustering when using k=10? \\
   * When for computing the accuracy of 10 classes, first you need to define what is the label of each cluster. Do that by majority votes. In theory, you may get two or more clusters with the same labels. We will ignore that for now. Meaning that you don't need to bother in case there are two clusters with the same label according to the majority vote. Simply count what is the fraction of 'other' digits in each cluster.
2. What is the optimal k?
3. How do the results changes if you first run PCA?

# Compute projection (10 points)
**Notice:** No code required in this section.

Given the next PCA projection matrix
$\begin{pmatrix}
1 & 1\\
2 & -1
\end{pmatrix}$
And the correspoding egienvalues
$(5, -1)$

Compute the projection to one dimension of the next two vectors:
\begin{pmatrix}
1 & 3
\end{pmatrix}
\begin{pmatrix}
3 & 7
\end{pmatrix}

In [ ]:
# Answer here and show your computations (no code)

*Your answers here*